In [5]:
import cv2 
import numpy as np
import random



def preprocessing(raw , scaler =  3.3324 ):
    h, w, _ = raw.shape
    
    y=0
    x=0
    h=4800
    w=raw.shape[1]
    img = raw[y:h, x:w]
    img = cv2.rotate(img, cv2.ROTATE_180)

    img  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    new_width = int(img.shape[1] / scaler )
    new_height = int(img.shape[0] / scaler)
    resized_image = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    

    return resized_image






def remove_outliers(data , x_data , k=1.5):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1

    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr

    y_centre = data[(data >= lower_bound) & (data <= upper_bound)]
    x_centre = x_data[(data >= lower_bound) & (data <= upper_bound)]

    return x_centre , y_centre


#iterates in x and y in blocks 10-pixels each and finds the blocks with the minimum sum and returns them in an array


def membrane_detection(denoised):
    height , width  = denoised.shape

    block_size = 10

    y_center_array = []
    x_center_array = []
    for x in range(0, width, block_size):
        y = 0  
        block_sums = []

        while y < height:
            block_sum = np.sum(denoised[y:y+block_size, x:x+block_size])
            block_sums.append(block_sum)
            y += block_size

        max_index = np.argmax(block_sums)

        y_center = (max_index * block_size) - 5

        y_center_array.append(y_center)
        x_center_array.append(x)


    x_center_array , y_center_array = remove_outliers(np.array(y_center_array) , np.array(x_center_array))


    return y_center_array , x_center_array


#takes an image and y and x coords as an arrays then sets every pixel under that line to black
def crop_line(denoised , y_center_array , x_center_array):
    
    height , width  = denoised.shape
    
    m, b = np.polyfit(x_center_array,y_center_array, 1)
    if(m<=0.1 and m >= -0.1):
        x = np.arange(width)
        y = m * x + b
        mask = np.repeat(y[np.newaxis, :], height, axis=0) < np.arange(height)[:, np.newaxis]

        denoised = np.where(~mask, denoised, 0)
        
        return denoised , m , b
        
    else:
        y_coord = int(np.percentile(y_center_array,75))
        denoised[y_coord+1:, :] = 0
        
        return denoised , 0 , int(b)
        
    

def draw_line(image, m, b):
    _ , w = image.shape[:2]
    x1 = 0
    y1 = int(m * x1 + b)
    x2 = w - 1
    y2 = int(m * x2 + b)
    image_with_line = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.line(image_with_line, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
    return image_with_line



#takes and image and x and y coords and returns the
#  first non-black pixel above it and None if Not found
def non_black_pixel(image, x, y): 
    row = int(y) - 1
    while row >= 0 and image[row, x] != 255:
        row -= 1   
    if row >= 0:
        return (x, row)
    else:
        return None
    
    
def denoising(img , lower = 25 , higher = 202 , blur = 5):
    
    denoised = cv2.fastNlMeansDenoising(img, h=10, templateWindowSize=7, searchWindowSize=21)

    for _ in range(blur):
        denoised = cv2.GaussianBlur(denoised,(5,5),0)

    
    lower = np.array([lower])
    upper = np.array([higher])

    mask = cv2.inRange(denoised, lower, upper)
    denoised = cv2.bitwise_and(denoised, denoised, mask=mask)
    

    return denoised


#takes an image , m and b for the membrane line equation , and returns the distance of the biofilm from each pixel 
def calc_dist(img , m , b , raw ):
    dists = []
    dists_to_compare = []
    
    for _ in range(1000):
        x_r = random.randint(5,img.shape[1]-5)
        y_r = m * x_r + b
        coords_r = non_black_pixel(img , x_r , int(y_r))
                                   
        if coords_r is not None:
            _ , y_r2 = coords_r
            dists_to_compare.append(y_r2)

        
    for x in range(img.shape[1]):S
        y = m * x + b
        coords = non_black_pixel(img , x , int(y) )

        if coords is not None:
            x1 , y1 = coords
            _ , y_w = coords
            coords2 = non_black_pixel(img , x1 , int(y1) )

            if (coords2 is not None):
                x2 , y2 = coords2
                y_w = min([y_w,y2], key=lambda x:abs(x-np.array(dists_to_compare).mean()))
                coords3 = non_black_pixel(img , x2 , int(y2) )

                if(coords3 is not None):
                    _ , y3 = coords3
                    y_w = min([y_w,y3], key=lambda x:abs(x-np.array(dists_to_compare).mean()))

                     
            dists.append(y - y_w)
            
    dists = np.array(dists)
    q1, q3 = np.percentile(dists, [25, 75])
    iqr = q3 - q1 
    
    lower_bound = q1 -  1.5*iqr
    upper_bound = q3 +  1.5*iqr
    
    dists = dists[(dists >= lower_bound) & (dists <= upper_bound)]
    
    img = draw_line(raw , m, b - dists.mean())
            

    return dists,  img


def split_image_cv(img):

    height, width = img.shape[:2]

    # Calculate the number of splits
    num_splits = width // 400

    # Split the image
    images = []
    for i in range(num_splits):
        left = i * 400
        right = (i + 1) * 400
        if right > width:
            right = width
        split_img = img[:, left:right]
        images.append(split_img)

    # Return the list of split images
    return images


In [6]:
import cv2
import numpy as np
import os
from tqdm import tqdm


if __name__ == '__main__':
    #specify source folder path
    path = r'C:\Users\omaro\Desktop\PatternSubmission\in'

    file_count = 1
    for i in tqdm(os.listdir(path)):

        file_path  = r'C:\Users\omaro\Desktop\PatternSubmission\out'
        file_path2  = r'C:\Users\omaro\Desktop\PatternSubmission\out'
        



        
        
        raw = cv2.imread(os.path.join(path, i))


        if not os.path.exists(file_path):
            os.mkdir(file_path)


        raw = preprocessing(raw)
        cv2.imwrite(file_path + '\original.png' , raw)
        
        images = split_image_cv(raw)
        i = 0
        for image in images:
            cv2.imwrite(file_path +  '\preprocessed' + str(i) + '.png' , image)
            i+=1
            img = denoising(image ,  lower = 22)
            cv2.imwrite(file_path +  '\img_preprocessed' + str(i) + '.png' , img)

            y,x = membrane_detection(img)
            img , m , b  = crop_line(img , y , x)
            cv2.imwrite(file_path +  '\membrane_c_preprocessed' + str(i) + '.png' , img)

            T1 = 80 
            T2 = 50
            img[img!=0] = 255
            edge = cv2.Canny(img, T1, T2 , apertureSize=3)



            cv2.imwrite(file_path +  '\edge_preprocessed' + str(i) + '.png' , edge)

            average_dists , edge_with_avg_line = calc_dist(edge , m , b , image)

            #CONCATENTE THE AVERAGE VALUE TO BE THE LAST VALUE IN THE FILE, THIS FILE CONTAINCS THE THICKNESS FOR EACH PIXEL
            cv2.imwrite(file_path +  '\membrane_c_final' + str(i) + '.png' , edge_with_avg_line)




            file_count+=1

            del img
            del edge



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.69s/it]
